### Import Libraries

In [2]:
from wrapper_functions import *
import time

### Read in Data

In [3]:
training2010 = pd.read_csv('../data/merged_wp_census_data_NY_081122.csv')
county_adj = pd.read_csv('../data/countyadj_NY.csv', index_col = 0)
models = ['acs', 'pep', 'worldpop']

# Default Configs

In [4]:
# MCMC configs.
mcmc_step_size=0.1 # @param
mcmc_sample_size=500 # @param
mcmc_num_steps=10_000 # @param
mcmc_burnin=2_500 # @param
mcmc_nchain=10 # @param
mcmc_seed=0 # @param

DEFAULT_MCMC_CONFIG = dict(step_size=mcmc_step_size, 
                           num_steps=mcmc_sample_size, 
                           burnin=mcmc_burnin, 
                           nchain=mcmc_nchain, 
                           seed=mcmc_seed)

## Bayesian Model Averaging

A Bayesian ensemble model where ensemble weights $w_k's$ are parameterized by Gaussian process priors:

$y \sim N(\mu(x), \sigma^2)$ 

$\mu(x) = \sum_{k=1}^K w_k(x) * m_k(x) \quad$  where $\{m_k\}_{k=1}^K$ are base model predictions.

$w(x) = softmax(f(x)) \qquad\;\;\;$ where $w=[w_1, \dots, w_K]$ and $f=[f_1, \dots, f_K]$

$f \stackrel{i.i.d.}{\sim} GaussianProcess(0, k)$


### Build Model

### Run MCMC

In [50]:
#### Makes the MCMC sampler (with the log probability function!)
def prepare_mcmc_CAR():
  """prepares the initial state and log prob function"""
  Q = (1/tau2)*(np.diag(county_adj.sum(axis=1)) - rho*county_adj)
  Q = tf.constant(Q, dtype = tf.float32)
  init_state = tf.constant(np.array([mv_normal_sample(precision_matrix = Q, num_models = 3) for i in range(nchain)]),
                           dtype = tf.float32)

  return init_state, target_log_prob_fn_CAR

def run_mcmc_CAR(init_state: Optional[List[tf.Tensor]] = None,
             target_log_prob_fn: Optional[Callable[..., tf.Tensor]] = None, 
             model_dist: Optional[List[tfd.Distribution]] = None,      
             y: Optional[tf.Tensor] = None,
             sample_size: int = 500, 
             nchain: int = 10,             
             num_steps: int = 500, 
             burnin: int = 100, 
             step_size: float = .1, 
             seed: int = 0, 
             debug_mode: bool = False,
             **mcmc_kwargs):
  """Executes MCMC training for a given model posterior.
  
  Args:
    target_log_prob_fn: The log likelihood function of modle posterior.
      If not provided, then a default set of (init_state, target_log_prob_fn)
      will be generated by `prepare_mcmc`.    
    init_state: The initial states to the MCMC sampler, a list of tf.tensors 
      with shape (num_chains, num_variables). If not provided, then a default 
      set of (init_state, target_log_prob_fn) will be generated by 
      `prepare_mcmc`.
    model_dist: The model posterior distribution to be used by `prepare_mcmc`. 
      Must be provided if init_state or target_log_prob_fn is None.
    y: The output variable with shape (batch_size, ) to be used by 
      `prepare_mcmc`.  Must be provided if init_state or target_log_prob_fn is
       None.
    sample_size: The number of the final MCMC samples to return after thinning
      the gathered MCMC samples.
    n_chain: The number of MCMC chain in sampling.
    num_steps: The number of total MCMC samples to generate.
    burnin: The length of the burn-in period for MCMC warmup.
    seed: The random seed for MCMC sampling.
    debug_mode: If True. also return the original unmixed samples.
    **mcmc_kwargs: Additional keyword arguments to pass to the low-level MCMC
      function.

  Return:
    mixed_samples: A list of posterior samples with shapes [sample_size, ...]
      for each variable in the model posterior. 
    sampler_stat: diagnostic statistics of the MCMC chain, which contains 
      the step size and the proposal acceptance of each HMC step.
  """
  # Prepares initial states and model log likelihoods for MCMC.
  if init_state is None or target_log_prob_fn is None:
    # By default, sample first parameter of a two-parameter model (W, y).
    init_state, target_log_prob_fn = prepare_mcmc_CAR(
        model_dist, y, nchain=nchain)
  else:
    nchain = init_state.shape[0]
    
  # Perform MCMC.
  chain_samples, sampler_stat = run_chain_CAR(
      init_state=init_state, 
      step_size=step_size,
      target_log_prob_fn=target_log_prob_fn,
      num_steps=num_steps, 
      burnin=burnin, 
      seed=seed,
      **mcmc_kwargs)
  # Clear tf.function cache.
  try:
    try:
      run_chain._stateful_fn._function_cache.clear()
    except:
      run_chain._stateful_fn._function_cache.primary.clear()
  except:
    print('no cache clearing')

  # Thinning.
  sample_size_per_chain = int(sample_size / nchain)
  sample_ids = np.linspace(
      0, num_steps-1, sample_size_per_chain).astype(int)
  chain_samples_thinned = chain_samples.numpy()[sample_ids]

  # Mix examples from different chains, 
  # Shape [param_dim_1, param_dim_2, num_mcmc_samples].
  mixed_samples = mix_chain_samples(chain_samples_thinned)

  # Check acceptance probability.
  p_accept = tf.math.exp(tfp.math.reduce_logmeanexp(
    tf.minimum(sampler_stat[-1], 0.)))
  print(f'Acceptance Ratio: {p_accept}')
  
  if debug_mode:
    return mixed_samples, chain_samples, sampler_stat
  return mixed_samples, sampler_stat

def run_chain_CAR(init_state: List[tf.Tensor], 
              step_size: float, 
              target_log_prob_fn: Callable[..., tf.Tensor], 
              num_steps: int = 500, 
              burnin: int = 100, 
              seed: int = 0,
              kernel_type: str = "hmc",
              step_adaptor_type: str = "simple"
              ) -> Union[List[tf.Tensor], Tuple[tf.Tensor]]:
  """Low-level function that runs MCMC sampling for a given model posterior.
  
  Args:
    init_state: The initial state for the MCMC sampler.
    step_size: The step size of a Hamiltonian Monte Carlo step.
    target_log_prob_fn: The log likelihood function for model posterior.
    num_steps: The number of total MCMC samples to return.
    burnin: The length of the burn-in period for MCMC warmup.
    seed: The random seed for MCMC sampling.
    kernel_type: Type of MCMC kernel to use, either ('hmc', 'nuts').
    step_adaptor_type: Type of MCMC kernel to use, one of 
      ('simple', 'dual_averaging').

  Returns:
    chain_state: Posterior sample from all MCMC chains.
    sampler stat: Sampling statistics, currently (step_size, acceptance ratio).
  """
  if kernel_type not in ('hmc', 'nuts'):
    raise ValueError(
        f"kernel_type {kernel_type} must be one of ('hmc', 'nuts').")

  if step_adaptor_type not in ('simple', 'dual_averaging'):
    raise ValueError(
        f"step_adaptor_type {step_adaptor_type} must be one of "
        "('simple', 'dual_averaging').")

  def trace_fn(_, pkr): 
    if kernel_type is 'hmc':
      step_size = pkr.inner_results.accepted_results.step_size
    else:
      step_size = pkr.inner_results.step_size

    return (step_size, pkr.inner_results.log_accept_ratio)

  if kernel_type is 'hmc':
    kernel = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob_fn,
        num_leapfrog_steps=5,
        step_size=step_size)
    step_adaptation_kwargs = dict()
  else:
    kernel = tfp.mcmc.NoUTurnSampler(
        target_log_prob_fn=target_log_prob_fn,
        step_size=step_size)
    step_adaptation_kwargs = dict(
        step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(
            step_size=new_step_size),
        step_size_getter_fn=lambda pkr: pkr.step_size,
        log_accept_prob_getter_fn=lambda pkr: pkr.log_accept_ratio,)

  if step_adaptor_type is 'simple':
    kernel = tfp.mcmc.SimpleStepSizeAdaptation(
      inner_kernel=kernel, 
      num_adaptation_steps=burnin)
  else:
    kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
      inner_kernel=kernel,
      num_adaptation_steps=burnin,
      target_accept_prob=0.75,
      **step_adaptation_kwargs)

  print(num_steps)
  # Execute sampling.
  chain_state, sampler_stat = tfp.mcmc.sample_chain(
      num_results=num_steps,
      num_burnin_steps=burnin,
      current_state=init_state,
      kernel=kernel,
      trace_fn=trace_fn,
      seed=seed)
    
  return chain_state, sampler_stat

<>:135: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:142: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:158: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:135: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:142: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:158: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Admin-Dell\AppData\Local\Temp\ipykernel_31496\2552605282.py:135: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if kernel_type is 'hmc':
C:\Users\Admin-Dell\AppData\Local\Temp\ipykernel_31496\2552605282.py:142: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if kernel_type is 'hmc':
C:\Users\Admin-Dell\AppData\Local\Temp\ipykernel_31496\2552605282.py:158: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if step_adaptor_type is 'simple':


In [6]:
## This function was taken from online
# Generate samples from a multi-variate normal distribution with provided precision matrix WITHOUT inverting
def mv_normal_sample(mu=0, precision_matrix=None, num_models=1):

    # Precision matrix must be a square matrix
    assert precision_matrix.shape[0] == precision_matrix.shape[1], 'Precision matrix must be a square matrix'

    dim = precision_matrix.shape[0]

    chol_U = scipy.linalg.cholesky(precision_matrix, lower=False)

    # Create num_models iid standard normal vectors
    z_vector_matrix = np.random.normal(loc=0, scale=1, size=[num_models, dim])

    # Sample from the MV normal with precision matrix by solving the Cholesky decomp for each normal vector
    samples = np.squeeze(np.array(
        [scipy.linalg.solve_triangular(a=chol_U, b=z_vector_matrix[i, :], unit_diagonal=False) + mu for i in
         range(num_models)]))

    return (np.transpose(samples))

In [7]:
# init state should is the CAR values at each county
nchain = 2
tau2 = 100
rho = 0.3

Q = (1/tau2)*(np.diag(county_adj.sum(axis=1)) - rho*county_adj)
Q = tf.constant(Q, dtype = tf.float32)

init_state = tf.constant(np.array([mv_normal_sample(precision_matrix = Q, num_models = 3) for i in range(nchain)]),
                        dtype = tf.float32)
init_state.shape

TensorShape([2, 62, 3])

In [47]:
def target_log_prob_fn_CAR(phi):
    print('sup')
    
    Q = (1/tau2)*(np.diag(county_adj.sum(axis=1)) - rho*county_adj)
    Q = tf.constant(Q, dtype = tf.float32)
        
    ll = tf.Variable(0.)
    for chain in range(phi.shape[0]):
        # (1) Prob of the CAR random effect values
        ll_chain = -0.5*tf.reduce_mean(tf.linalg.diag_part(
            tf.linalg.matmul(phi[chain,:,:],tf.linalg.matmul(Q, phi[chain,:,:]), transpose_a = True))) 
        ll = ll + ll_chain
    
    # add in determinant values
    log_det = tf.constant(np.linalg.slogdet(Q)[1], dtype = tf.float32)
    ll = ll + 0.5*phi.shape[0]*len(models)*log_det
    
    # get exponentiated values and sum across models
    exp_phi = tf.math.exp(phi)
    exp_phi_rows = tf.reduce_sum(exp_phi, 2)
    
    # get model weights and calculate mean estimate
    u = exp_phi/exp_phi_rows[...,None]
      
    tmp = training2010[models].values*u
    n = tf.reduce_sum(tmp, axis = 2)
    
    # update the log likelihood 
    ll = ll + tf.reduce_sum([np.sum(training2010['census']*np.log(n[chain,:]) - n[chain,:]) for chain in range(phi.shape[0])])
    
    print(ll)
    return(ll)

In [9]:
len(tf.config.list_physical_devices('GPU'))

0

In [34]:
mcmc_config = DEFAULT_MCMC_CONFIG.copy()

In [32]:
#mcmc_config.update(dict(burnin = 2500, num_steps = 10000, nchain = 5))
mcmc_config.update(dict(burnin = 2500, num_steps = 10000))
mcmc_config

{'step_size': 0.1, 'num_steps': 10000, 'burnin': 2500, 'nchain': 10, 'seed': 0}

Seems like this takes ~2.6s per num_steps
Took 265s for num_steps = 100
Took 51s for num_steps = 50
102s for num_steps = 100
262s for num_steps = 100 test
12s for num_steps = 100 the improved test
and 16m for num_steps = 10000 and nchain = 5! Great

In [54]:
from wrapper_functions import *
#from my_functions import *

In [13]:
run_mcmc

<function wrapper_functions.run_mcmc(init_state: Optional[List[tensorflow.python.framework.ops.Tensor]] = None, target_log_prob_fn: Optional[Callable[..., tensorflow.python.framework.ops.Tensor]] = None, model_dist: Optional[List[tensorflow_probability.python.distributions.distribution.Distribution]] = None, y: Optional[tensorflow.python.framework.ops.Tensor] = None, sample_size: int = 500, nchain: int = 10, num_steps: int = 500, burnin: int = 100, step_size: float = 0.1, seed: int = 0, debug_mode: bool = False, **mcmc_kwargs)>

So with wrapper functions but my "run_chain" this does not work
with wrapper functions but my "run_chain" and "run_mcmc" this works
with wrapper functions but my "run_mcmc" this doesn't work

So these both together are the problem? What? How could this be possible? I guess they both changed in some way? How? How did these change

In [55]:
t0 = time.perf_counter()
CAR_samples, chain_samples, sampler_stat = run_mcmc_CAR(init_state=init_state,
                             target_log_prob_fn=target_log_prob_fn_CAR,
                             debug_mode = True,
                             **mcmc_config)  
print(time.perf_counter() - t0)

500
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968160.0, shape=(), dtype=float32)
sup
tf.Tensor(485968160.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968200.0, shape=(), dtype=float32)
sup
tf.Tensor(485968160.0, shape=(), dtype=float32)
sup
tf.T

tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968260.0, shape=(), dtype=float32)
sup
tf.Tensor(485968300.0, shape=(), dtype=float32)
sup
tf.Tensor(485968300.0, shape=(), dtype=float32)
sup
tf.Tensor(48

KeyboardInterrupt: 

Saving and loading Python objects with Pickle

In [47]:
import pickle
with open('../data/CAR_samples_with_mat_determinant.pickle', 'wb') as results_file:
  pickle.dump(CAR_samples, results_file)

In [33]:
with open(r"../data/CAR_samples.pickle", "rb") as input_file:
     CAR_samples2 = pickle.load(input_file)

In [21]:
mcmc_nchain = 10

# I can just run this first to get the code working. This way I can skip editing posterior_inference
# and the prepare_mcmc and just focus on the log probability function
CAR_samples, _ = run_mcmc(init_state=init_state,
                             target_log_prob_fn=target_log_prob_fn,
                             **mcmc_config)  

bma_gp_w_samples = run_posterior_inference(model_dist=bma_prior, 
                                           model_config=bma_model_config,
                                           Y=Y_train, 
                                           map_config=map_config,
                                           mcmc_config=mcmc_config)


bma_joint_samples = make_bma_samples(X_test1, Y_test, base_preds_test, 
                                     bma_weight_samples=bma_gp_w_samples[0],
                                     bma_model_config=bma_model_config, 
                                     n_samples=bma_n_samples_eval, 
                                     seed=bne_seed,
                                     y_samples_only=False)



Running MAP:	13129634283520.0...13028767744.0...12917108736.0...12837866496.0...12796110848.0...12773912576.0...12757137408.0...12761999360.0...12907206656.0...13081935872.0...Done.
Running MCMC:	Acceptance Ratio: 0.6058999300003052


In [48]:
models

['acs', 'pep', 'worldpop']

In [49]:
CAR_ensemble_weights = tf.reduce_mean(CAR_samples[0], axis = 2)

weights_dict = {
    "acs": CAR_ensemble_weights[:,0],
    "pep": CAR_ensemble_weights[:,1],
    "worldpop": CAR_ensemble_weights[:,2]
}

color_weights = make_color_norm(
    list(weights_dict.values())[1],   
    method="percentile")

In [50]:
# get exponentiated values and sum across models
exp_phi = tf.math.exp(CAR_ensemble_weights)
exp_phi_rows = tf.reduce_sum(exp_phi, 1)

# get model weights and calculate mean estimate
u = exp_phi/exp_phi_rows[...,None]


In [51]:
norm_weights_dict = {
    "acs": u[:,0],
    "pep": u[:,1],
    "worldpop": u[:,2]
}

color_norm_weights = make_color_norm(
    list(norm_weights_dict.values())[1],   
    method="percentile")

In [56]:
import plotly
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
# df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
#                    dtype={"fips": str})
import matplotlib.colors as colors

import plotly.express as px

In [53]:
for model_name in models:
    output = pd.DataFrame(np.column_stack([training2010[["GEOID"]], weights_dict[model_name]]))
    output = output.set_axis(['GEOID', model_name], axis=1)
    output[model_name] = output[model_name].astype(float)
    fig = px.choropleth_mapbox(output, geojson=counties, locations='GEOID', color=model_name,
                           color_continuous_scale="Viridis",
                           #range_color=(0.05,0.07),
                           mapbox_style="carto-positron",
                           #featureidkey="properties.MWS_ID",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

NameError: name 'counties' is not defined

In [54]:
for model_name in models:
    output = pd.DataFrame(np.column_stack([training2010[["GEOID"]], norm_weights_dict[model_name]]))
    output = output.set_axis(['GEOID', model_name], axis=1)
    output[model_name] = output[model_name].astype(float)
    fig = px.choropleth_mapbox(output, geojson=counties, locations='GEOID', color=model_name,
                           color_continuous_scale="Viridis",
                           #range_color=(0.05,0.07),
                           mapbox_style="carto-positron",
                           #featureidkey="properties.MWS_ID",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'unemp':'unemployment rate'}
                          )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

NameError: name 'counties' is not defined

! conda install geopandas==0.3.0
! conda install pyshp==1.2.10
! conda install shapely==1.6.3